In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# Supress Warnings
import warnings
warnings.filterwarnings('ignore')
import random



In [ ]:
train=pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv')
train.head()

In [ ]:
test=pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv')
test.head()

In [ ]:
def eda(df):
  print(df.info())
  print("****"*10)
  print('shape of dataset : ',end=" ")
  print(df.shape)
  print("****"*10)
  print(df.describe().T)
  print("****"*10)
  print(df.isnull().sum())
  print("****"*10)
  print('Total Missing Values : ')
  print(df.isnull().sum().sum())
  print("****"*10)


In [ ]:
print('EDA on train dataset: ')
eda(train)

In [ ]:
print('EDA on test dataset: ')
eda(train)

In [ ]:
Features=[col for col in train.columns if col not in ['id','target']]

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
pie = ax.pie([len(train), len(test)],
             labels=["Train dataset", "Test dataset"],
             colors=["salmon", "teal"],
             textprops={"fontsize": 15},
             autopct='%1.1f%%')
ax.axis("equal")
ax.set_title("Dataset length comparison", fontsize=18)
fig.set_facecolor('white')
plt.show();

In [ ]:
plt.rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')

# for Interactive Shells
from IPython.display import display


In [ ]:
train.isnull().sum().sort_values(ascending = False).to_frame().head(70).rename({0:'Counts'}, axis = 1).T.style.background_gradient('crest')


In [ ]:
test.isnull().sum().sort_values(ascending = False).to_frame().head(70).rename({0:'Counts'}, axis = 1).T.style.background_gradient('crest')


In [ ]:
train.describe().style.background_gradient(cmap='bone_r')

In [ ]:
test.describe().style.background_gradient(cmap='bone_r')

In [ ]:
train.corr()[['target']].T.style.background_gradient('copper_r')

In [ ]:
df_plot = ((train -train.min())/(train.max() - train.min()))
fig, ax = plt.subplots(4, 1, figsize = (25,25))
sns.boxplot(data = df_plot.iloc[:, 1:30], ax = ax[0])
sns.boxplot(data = df_plot.iloc[:, 30:60], ax = ax[1])
sns.boxplot(data = df_plot.iloc[:, 60:90], ax = ax[2])
sns.boxplot(data = df_plot.iloc[:, 90:120], ax = ax[3])


In [ ]:
nrows = 20
ncols = 6
i = 0
fig, ax = plt.subplots(nrows, ncols, figsize = (25,75))
for row in range(nrows):
    for col in range(ncols):
        sns.histplot(data = train.iloc[:, i], bins = 50, ax = ax[row, col], palette  = 'bone_r').set(ylabel = '')
        i += 1


In [ ]:
df = pd.concat([train.drop(["id", "target"], axis=1), test.drop("id", axis=1)], axis=0)
columns = df.columns.values

cols = 4
rows = len(columns) // cols + 1

fig, axs = plt.subplots(ncols=cols, nrows=rows, figsize=(16,130), sharex=False)

plt.subplots_adjust(hspace = 0.3)
i=0

for r in np.arange(0, rows, 1):
    for c in np.arange(0, cols, 1):
        if i >= len(columns):
            axs[r, c].set_visible(False)
        else:
            hist1 = axs[r, c].hist(train[columns[i]].values,
                                   range=(df[columns[i]].min(),
                                          df[columns[i]].max()),
                                   bins=40,
                                   color="deepskyblue",
                                   edgecolor="black",
                                   alpha=0.7,
                                   label="Train Dataset")
            hist2 = axs[r, c].hist(test[columns[i]].values,
                                   range=(df[columns[i]].min(),
                                          df[columns[i]].max()),
                                   bins=40,
                                   color="palevioletred",
                                   edgecolor="black",
                                   alpha=0.7,
                                   label="Test Dataset")
            axs[r, c].set_title(columns[i], fontsize=12, pad=5)
            axs[r, c].set_yticks(axs[r, c].get_yticks())
            axs[r, c].set_yticklabels([str(int(i/1000))+"k" for i in axs[r, c].get_yticks()])
            axs[r, c].tick_params(axis="y", labelsize=10)
            axs[r, c].tick_params(axis="x", labelsize=10)
            axs[r, c].grid(axis="y")
            axs[r, c].legend(fontsize=13)
                                  
        i+=1
#plt.suptitle("Feature values distribution in both datasets", y=0.99)
plt.show()

In [ ]:
print("Features with the less amount of unique values:")
train.drop(["id", "target"], axis=1).nunique().sort_values().head(5)


In [ ]:
print("Features with the less amount of unique values:")
test.drop("id", axis=1).nunique().sort_values().head(5)


In [ ]:
df = train.drop("id", axis=1).corr().round(5)

# Mask to hide upper-right part of plot as it is a duplicate
mask = np.zeros_like(df)
mask[np.triu_indices_from(mask)] = True

# Making a plot
plt.figure(figsize=(32,24))
ax = sns.heatmap(df, annot=False, mask=mask, cmap="RdBu", annot_kws={"weight": "bold", "fontsize":13})
ax.set_title("Feature correlation heatmap", fontsize=17)
plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
         rotation_mode="anchor", weight="normal")
plt.setp(ax.get_yticklabels(), weight="normal",
         rotation_mode="anchor", rotation=0, ha="right")
plt.show()

In [ ]:
print(': )')